# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [76]:
def priest(sex, age, respiratoryRate, O2Sat, heartRate,systolicBP,temp,alertness, inspiredO2, pstatus): 
    """This function computes the risk percentage that should be 
        returned as a numeric value between
        0 and 1.
        
        Args:
            sex (str): 
            the gender of the patient.
            
            age (int): 
            the age of the patient.
            
            respiratoryRate (int) : 
            the number of breaths per minute 
            
            O2Sat (float): 
            percentage of Oxygen saturation (given between 0 and 1)
            
            heartRate (int):
            Heart Rate of patient in beats per minute
            
            systolicBP (float):
            Systolic BP in mmHG
            
            alertness (string):
            Alertness of the patient
            
            inspiredO2 (string):
            Inspired Oxygen
            
            pStatus (string):
            Performance Status of the patient
            
            
        Returns:
            float : Numeric value between 0 and 1
        
        Raises:
            Invaid Value Exceptions
    >>> priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.47
    >>> priest ('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
    0.9
    >>> priest('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no SELF-care')
    0.22
    """
    #---------------------------------------Sex------------------------------------------
    sexx = sex.lower()
    if sexx == "male":
        sex = 1
    elif sexx == "female":
        sex = 0
    else:
        raise Exception(" Enter only 'male' or 'female' in parameter : Sex")
    #-------------------------------------------Age-----------------------------------------
    try:
        if 16<=age<=49:
            age = 0
        elif 50<=age<=65:
            age = 2
        elif 66<=age<=80:
            age = 3
        elif 80<age:
            age = 4
    except ValueError:
        print("Invalid Input in Parameter: Age")

    #-----------------------------------------respiratoryRate---------------------------------
    try:
        if respiratoryRate < 9 :
            r = 3
        elif respiratoryRate in range(9,12):
            r = 1
        elif respiratoryRate in range(12,21):
            r = 0
        elif respiratoryRate in range(21,25):
            r = 2
        elif respiratoryRate > 24:
            r = 3
    except:
        print("Invalid Input in Parameter: Respiratory Rate")
    #------------------------------------O2Sat-------------------------------------------------
    try:
        if O2Sat>0.95:
            sat = 0
        elif 0.94 >= O2Sat >= 0.95:
            sat = 1
        elif 0.92 >= O2Sat >= 0.93:
            sat = 2
        elif 0.92 > O2Sat:
            sat = 2
    except:
        print("Invalid Input in Parameter: Oxygen Saturation")
    #----------------------------------Heart Rate-------------------------------------------------
    try:
        if heartRate < 41:
            hr = 3
        elif 41 <=  heartRate <= 50:
            hr = 1
        elif 51 <= heartRate <= 90:
            hr = 0
        elif 91 <= heartRate <= 110:
            hr = 1
        elif 111 <= heartRate <= 130:
            hr = 2
        elif 130 <= heartRate:
            hr = 3
    except:
        print("Invalid Input in Parameter: Heart Rate")
    #----------------------------------------Temperature---------------------------------------------
    try:
        if temp < 35.1:
            temp = 3
        elif 31.1 <= temp <= 36.0:
            temp = 1
        elif 36.1 <= temp <= 38.0:
            temp = 0
        elif 38.1 <= temp <= 39.0:
            temp = 1
        elif 39.0 < temp :
            temp = 2
    except:
        print("Invalid Input in Parameter: Temperature")
    #---------------------------------------Alertness----------------------------------------------
    at = 0
    if (alertness).upper() == "ALERT":
        at = 0
    elif (alertness).upper() == "CONFUSED OR NOT ALERT":
        at = 3
    else:
        raise Exception("Invalid Input in Parameter : Alertness, only allowed('alert or 'Confuse or not alert')")
    #----------------------------------------Inspired Oxygen---------------------------------------
    io = 0
    if (inspiredO2).upper() == "AIR":
        io = 0
    elif (inspiredO2).upper() == "SUPPLEMENTAL OXYGEN":
        io = 2
    else:
        raise Exception("Invalid Input in Parameter : Inspired Oxygen, only allowed('Air or 'Supplemental Oxygen')")
    #--------------------------------------Performance Status-------------------------------------
    ps = 0
    
    if (pstatus).upper() == "UNRESTRICTED NORMAL ACTIVITY":
        ps = 0
    elif (pstatus).upper() == "LIMITED STRENOUD ACTIVITY, CAN DO LIGHT ACTIVITY":
        ps = 1
    elif (pstatus).upper() == "LIMITED ACTIVITY, CAN SELF-CARE":
        ps = 2
    elif (pstatus).upper() == "LIMITED SELF-CARE":
        ps = 3
    elif (pstatus).upper() == "BED/CHAIR BOUND, NO SELF-CARE":
        ps = 4
    else:
        raise Exception("Invalid Input in Parameter : Performance Status")
  #---------------------------------------------------Value and result---------------------------------------------   
    try:
        formula = sex + age + r + sat + hr + temp + at + io + ps
    except:
        return("Result is not generated")
    
    
    if formula in range (0,2):
        return(0.1)
    elif formula in range (2,4):
        return(0.2)
    elif formula == 4:
        return(0.3)
    elif formula == 5:
        return(0.9)
    elif formula ==6:
        return(0.15)
    elif formula ==7:
        return(0.18)
    elif formula ==8:
        return(0.22)
    elif formula==9:
        return(0.26)
    elif formula ==10:
        return(0.29)
    elif formula ==11:
        return(0.34)
    elif formula == 12:
        return(0.38)
    elif formula == 13:
        return(0.46)
    elif formula ==14:
        return(0.47)
    elif formula ==15:
        return(0.49)
    elif formula ==16:
        return(0.55)
    elif formula in range(17,26):
        return(0.59 + " to " + 0.88)
    elif formula >=26:
        return(0.99)
   

In [77]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.47
ok
Trying:
    priest ('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
Expecting:
    0.9
ok
Trying:
    priest('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no SELF-care')
Expecting:
    0.22
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [79]:
import requests
import json
def find_hospitals(age, sex, risk):
    """Based on a patient's risk score, age, and sex, 
    this function determines the nearest facility to take them in case of an emergency by using a REST web service.
    >>> find_hospitals(40, "female", 0.3)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospitals(25, "male", 0.8)
    'Emory Dunwoody Medical Center'
    >>> find_hospitals(53,"FEMALE",0.3)
    'Select Specialty Hospital - Northeast Atlanta'
    """
    
    response = requests.get(f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}')
    hospital_name = response.json()['hospital']
    return (hospital_name)

In [80]:
import doctest
doctest.run_docstring_examples(find_hospitals, globals(), verbose=True)

Finding tests in NoName
Trying:
    find_hospitals(40, "female", 0.3)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospitals(25, "male", 0.8)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospitals(53,"FEMALE",0.3)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [81]:
from bs4 import BeautifulSoup

In [85]:
def get_address(hospital_name):
    """ Whenever we find out which hospital to approach in an emergency, we should also find out the address 
   of the hospital. This function will print the address of the hospital from the website that has the list 
   of hospitals and their addresses.
    """ 
    response = requests.get('https://officialusa.com/stateguides/health/hospitals/georgia.html')

    soup = BeautifulSoup(response.content, 'html.parser')

    hospitals_html_table = soup.find("table", {"id": "myTable"}).find_all('tr')
    hospitals_html_table.pop(0)  # Remove table header
    hospital_data = {}
    for hospital in hospitals_html_table:
        temp = hospital.text.split('\n')
        if temp[1].lower() == 'atlanta':
            if hospital not in hospital_data:
                hospital_data[temp[2]] = temp[4]

    if hospital_name.upper() in hospital_data:
        print (hospital_data[hospital_name.upper()])
    else:
        raise KeyError("Not found")

## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [83]:
def process_people(file_path):
    """This function returns a dictionary which has all the parameters as follows {patient_number:[sex,age,breath,o2sat,heart,systolictemp,alertness,inspired]}
    
    """
    with open(file_path) as pd:
        people_data = pd.readlines()
        people_data.pop(0)

    people_dict = {}
    for person in people_data:
        temp = [x.strip() for x in person.split('|')]
        people_dict[temp[0]] = temp[1:]

    return people_dict

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [84]:
import json
json_file = "people_results.json"
with open("../data/"+json_file) as file:
    json_data = json.loads(file.read())
    peoples_results = process_people('../data/people.psv')
    if json_data == peoples_results:
        print("Both are Same")
       

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [10]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 no



    
OK. We can wait.

